# Introducción

- El "dataset" usado para la competición se encuentra en https://www.kaggle.com/competitions/diamonds-part-datamad0122/. Es parte de una competición cerrada sobre diamantes con sus características. La "target" es el precio de los diamantes

- El "dataset" con los precios reales, usado para obtener la mejor predicción del precio, se encuentra aquí: https://www.kaggle.com/datasets/swatikhedekar/price-prediction-of-diamond

- La fuente original de los dos anteriores es: "Tiffany & Co's snapshot pricelist from 2017"

- El "dataset" con las imágenes de los diamantes y sus características, se encuentra aquí: https://www.kaggle.com/datasets/harshitlakhani/natural-diamonds-prices-images

- La fuente original es esta, se ha obtenido con "webscrapping": https://capitalwholesalediamonds.com/

- Se tiene como objetivo inicial de este proyecto superar la cifra de la predicción ganadora de la competición

- El objetivo secundario es, mediante un segundo modelo que saque las características de un diamante a partir de su foto, obtener los datos necesarios para predecir su precio. Se usa un tercer modelo en su lugar para este fin, parecido al de la competición, pues no todas las variables necesarias para la predicción se pueden obtener del "dataframe" con los datos de los diamantes de las fotos. Además, la variable "target" del original es más precisa, pues no se ha escalado

- Con el modelo que obtiene las características de los diamantes a partir de una foto y el modelo que predice los precios se crea, además, una "app" de Streamlit que les saca partido: https://rogerperello-machine-learning-diamon--streamlitfilesmain-bke5k8.streamlit.app/

# EDA

- Es el análisis exploratorio de los datos de competición. Como las variables son las mismas, los cambios detectados posibles se extrapolan para el "dataset" original

- Se lleva a cabo en el "notebook" homónimo


## Observaciones iniciales

- Se adaptan los nombres de las columnas para que sean legibles

- No se detectan nulos

- Se encuentran duplicados, pero como todos tienen su propia identificación se asume que cada diamante es único y deben conservarse

- Se nota que hay diamantes con 0 milimetros de altura, anchura y/o profundidad

## Cambios probados

- Se detectan una serie de características:

1) Varias columnas tienen unos pocos valores atípicos extremadamente altos ["depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)"]

2) Como se ha anticipado, varias filas tienen un cero en todas las variables relacionadas con el tamaño de los diamantes, sin contar el peso ["lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

3) Dos columnas con una alta correlación negativa tienen unos pocos "outliers" que destacan moderadamente y que, además, son compartidos ["depth (percentage)" y "table (percentage)"].

4) Si se borrasen las filas con ceros del punto 2, todavía quedaría un cero en una fila para el "lenght". Como los diamantes son prácticamente circulares, "lenght" y "width" son casi idénticas ["lenght (millimeters)"].

5) Si se borrasen las filas con ceros del punto 2 y se solucionase el cero del punto 4, todavía quedaría un cero en una fila para "depth (millimeters)". Se descubre que la variable "depth (percentage)" se obtiene de dividir ese "depth (millimeters)" entre la media de "width" y "lenght" ["depth (millimeters)"].

6) Se detecta un "outlier" en "lenght" ["lenght (millimeters)"].

7) Varias columnas son asimétricas por la derecha. Con el logaritmo, se centrarían esas distribuciones y deaparecerían "outliers" ["weight (carat)", "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

8) Existe un valor atípico en "weight". Sin embargo, imputarlo al valor máximo de su "boxplot" haría que resurgiese después de un escalado estándar ["weight (carat)"].

9) Dos columnas tienen una alta correlación negativa que se quiere conservar, y muchos "outliers" no compartidos. Se presume que la posición de esos "outliers" es la que determina la correlación ["depth (percentage)" y "table (percentage)"].

10) "depth (millimeters)" tiene valores atípicos. Esta variable, junto con las demás relacionadas con el tamaño, están tan correlacionadas que casi podría considerarse que son la misma variable escalada ["depth (millimeters)"].

11) Algunas variables tienen valores mucho más altos comparadas con el resto.

----------------------------------

- Con esa información, se prueban una serie de posibles cambios a llevar a cabo durante la fase de "feature_engineering". Si se aplica el conjunto de estos cambios, el "dataframe" queda libre de nulos completamente:

1) Borrado de "outliers" extremadamente altos ["depth (percentage)", "table (percentage)", "width (millimeters)", "depth (millimeters)"].

2) Borrado de filas que tienen 0 en todas las variables de tamaño ["lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

3) Borrado de los "outliers" compartidos moderadamente altos ["depth (percentage)" y "table (percentage)"].

4) Asignación del valor con 0 restante en "lenght" al "width" correspondiente ["lenght (millimeters)"].

5) Asignación del valor con 0 restante de "depth (millimeters)" a partir de una operación con el "lenght", el "width" y el "depth (percentage)" correspondientes ["depth (millimeters)"].

6) Asignación del "outlier" restante del "lenght" al "width" correspondiente ["lenght (millimeters)"].

7) Transformación con logaritmo ["weight (carat)", "lenght (millimeters)", "width (millimeters)" y "depth (millimeters)"].

8) Imputación al siguiente valor más alto ["weight (carat)"].

9) Imputación a los valores máximos y mínimos del "boxplot" ["depth (percentage)" y "table (percentage)"].

10) Neutralización de "outliers" con un modelo "ridge" ["depth (millimeters)"].

11) Escalado.

## Cambios no probados

- Se detectan una serie de características:

12) La variable "depth (percentage)" sale de dividir "depth (millimeters)" por la media de "lenght (millimeters)" y "width (millimeters)". Por tanto, si se hace la operación, el resultado debería coincidir con la columna de "depth (percentage)". Sin embargo, no es así. Habrá que probar durante el "feature engineering" si utilizar los valores calculados mejora la predicción.

13) Las columnas relacionadas con el tamaño tienen una correlación altísima. Quizá los resultados mejoren con borrarlas. Otras columnas tinen una correlación ínfima, cercana a 0, que podría considerarse irrelevante

14) Las filas con el valor máximo de "clarity quality" (7, variable discreta) no cambian en cuanto a las variables relacionadas con el tamaño si se comparan con las que tienen una "clarity quality" un punto por debajo (6). Quizá cabría imputar esos valores 7 de "clarity quality" al 6.


# Feature_engineering

- Se aplican los cambios probados y no probados consecutivamente intercalando múltiples modelos para ver 1) qué modelos mejoran con qué cambios, y 2) qué modelo es el adecuado para la optimización. Se trabaja con el "dataset" de competición, pero los resultados son extrapolables al original

- Se lleva a cabo en el "notebook" homónimo

## Selección de cambios

- Se reúnen los cambios apuntados en el EDA en subgrupos:

a) Borrado: 1, 2 y 3 (borrado parcial: solo 2)

b) Asignación: 4, 5, 6 (asignación parcial: solo 4 y 5)

c) Logaritmo: 7

d) Valor más alto de "weight": 8

e) Imputaciones "boxplot": 9

f) Imputaciones "ridge": 10

g) Escalado "MinMax": 11

h) Escalad "Standard": 11

i) Sustitución: 12

j) Descarte correlación altísima: 13 -> ningún modelo mejora

j) Descarte correlación ínfima: 13

k) Imputaciones "clarity quality": 14

--------------------------------

- Cada uno de los modelos mejora con los siguientes cambios:


---------- LinearRegression ----------

· Borrado

· Asignación

· Logaritmo

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Imputaciones "clarity quality"

---------- Ridge ----------

· Escalado "MinMax"

· Borrado

· Asignación

· Logaritmo

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Imputaciones "clarity quality"

---------- KNeighborsRegressor ----------

· Escalado "Standard"

· Borrado parcial

· Asignación

· Valor más alto de "weight"

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Descarte correlación ínfima

· Imputaciones "clarity quality"

---------- SVR ----------

· Escalado "Standard"

· Borrado

· Asignación

· Logaritmo

· Valor más alto de "weight"

· Imputaciones "boxplot"

· Imputaciones "ridge"

· Sustitución

· Descarte correlación ínfima

---------- DecisionTree ----------

· Borrado parcial

· Asignación parcial

---------- RandomForest ----------

· Borrado parcial

· Asignación parcial

· Sustitución

---------- XGBRegressor ----------

· Borrado parcial

· Asignación parcial


## Selección de modelos

- Estos son los resultados, una vez aplicados los cambios y tuneados algunos hiperparámetros, si se compara la predicción con el "y_test":

a) "LinearRegression" mejora su "rmse" de 0.182664 a 0.143851 (-21.24%)

b) "Ridge" mejora su "rmse" de 0.182826 a 0.144234 (-21.11%)

c) k vecinos mejora su "rmse" de 0.181632 a 0.121120 (-33.32%)

d) "SVR" mejora su "rmse" de 0.189328 a 0.100319 (-47.01%)

e) "DecisionTree" mejora su "rmse" de 0.130480 a 0.109705 (-15.92%)

f) "RandomForest" mejora su "rmse" de 0.094246 a 0.091972 (-2.41%)

g) "XGBRegressor" mejora su "rmse" de 0.090762 a 0.086893 (-4.26%)

- Se elige trabajar en la siguiente fase con "XGB". Es el mejor con una "rmse" de 0.086893, y, como dispone de muchos más hiperparámetros que los probados hasta ahora, probablemente tenga un margen considerable de optimización

- El orden, si "XGB" (1) no cumpliese o fuera necesario otro modelo, sería: 2) "RandomForest"; 3) "SVR"; 4) "DecisionTree"; 5) k vecinos; 6) regresión lineal; y 7) "ridge"

# Target_transformation

- Se confirma que la variable "target" del "dataframe" de competición se ha escalado, y no representa los precios reales de los diamantes

- En el "notebook" homónimo se investiga si es cierto que los dos "datasets" son la misma cosa y qué tipo de escalado se ha usado

- El escalado efectuado es el logaritmo. Sin embargo, se han perdido decimales en el proceso. Por tanto, hay que usar el "dataframe" de competición para competir, y el original para predecir precios reales con mayor concreción

- Se exporta el original procesado sin la columna "table", pues esa columna tampoco existe en el "dataframe" sobre las características de las fotos de diamantes

- Se exporta, también, con la columna "table" por si hace falta sacar la mejor predicción posible sin usar las fotos de los diamantes


# Model_optimization

- Donde se experimenta de forma extensiva con "XGB" y k vecinos con tal de obtener los mejores resultados posibles. El primer modelo entrenado que sale es el que se usa para la competición. El segundo y el tercero, con una columna menos y extraídos del "dataframe" original de diamantes, no el de competición, se usan para predecir los precios de los diamantes de las fotos (aquellos cuyos valores entran en el rango de los vistos durante el entrenamiento y los que no, respectivamente)

- Se lleva a cabo en el "notebook" homónimo

## Mejoras para competición

- Estos son los hiperparámetros que se añaden al modelo a cada ronda:

1) Se prueba sin modificación alguna para tener una base desde la que hacer comparaciones

2) Se prueba con los cambios de "feature engineering" (borrado parcial y asignación parcial) por la misma razón

3) Se obtiene un número de estimadores óptimo como punto de partida y se tunea el "learning rate" ("eta"). Una vez tuneado, se vuelve a buscar el número óptimo de estimadores

4) Se aplican "constaints" a la columna "weight", ya que solo crece con el precio

5) Se establecen un "subsample" y "colsample_bytree" de 0.8, que es el estándar recomendable, y se tunean "max_depth" y "min_child_weight"

6) Se tunea "gamma", parámetro de semiregularización

7) Se prueban dos opciones y se tunean: a) "sampling" combinado con "colsample_bytree", y b) "sampling_method='gradient_based" combinado con "colsample_bytree". Gana la opción "a"

8) Se tunean los hiperparámetros de regularización ("alpha", "lambda" y "max_delta_step"), que son los que ayudan a prevenir el "overfitting" y el "underfitting"

9) Se aumenta el número de árboles en paralelo, convirtiendo, a todos los efectos, el "xgboost" en un "random forest" con "booster"

10) Se reajusta el número de estimadores una vez más

### Resultado competición

- Se superan los puntos del ganador de la competición, con una "rmse" de 0.08506 contra 0.08617

## Mejoras para visualización A

- Se repite el mismo proceso que en el punto anterior, pero esta vez se le quita la columna "table" al "dataframe", y se usa el "dataset" original

- Esto se hace porque en el "dataframe" que almacena los datos sobre las fotos de los diamantes no existe esa columna. Por tanto, es una variable que no se puede obtener a partir de una foto. Ello obliga a disponer de un modelo a parte del de la competición para la "app". Se usa el "dataset" original porque los precios son más precisos. Se le aplica el logaritmo a la "target" para facilitar el proceso

### Resultado A

-  Es incluso mejor que el del anterior modelo, pero podría considerarse trampa porque se usa el "dataframe" original. Se guarda a parte con el fin único de extraer los precios reales de los diamantes de las fotos

- Sin embargo, se comprueba que la predicción no cambia si el diamante a predecir tiene variables por encima de los rangos vistos en el entrenamiento. Eso ocurre porque los modelos de árboles no son tan precisos en los extremos, y, como se ha visto en "feature_enginerring" y en el EDA, la "r2" es alta y hay varias variables con una correlación altísima


## Mejoras para visualización B

- Con tal de predecir diamantes cuyas características sean superiores o inferiores a las vistas durante el entrenamiento, se elige el siguiente modelo no "de árboles" visto durante en "feature_engineering": SVR

- Sin embargo, como el coste computacional de SVC es tan elevado, y se presume que será a aún mayor si se lleva a cabo un "stacking", se elige el siguiente de la lista: k vecinos

- A pesar de lo visto en el "feature_engineering", como se pretende usar el modelo solo para valores muy altos o bajos a la hora de predecir, interesa que vea los "outliers". Por tanto, no se le hace más tratamiento que el que ha recibido "xgboost"

### Resultado B

- La predicción con el "y_test" es peor, lo que confirma que "xgboost" trabaja mejor con los datos similares a los vistos durante el entrenamiento. Sin embargo, al subirlo a "Kaggle", se dispara. K vecinos es el complemento perfecto para predecir cuando los valores están fuera de los rangos vistos en el "train"


## "Stacking"

- Se intenta combinar las bondades de ambos modelos con un "stacking". Para ello se usa un tercer modelo sencillo de regresión lineal

- El modelo resultante no predice tan bien como "xgboost" (A) ni tan mal como kvecinos (B), pero puede predecir a partir de valores que no ha visto en el entrenamiento

- Por tanto, este es el modelo que se usará para la predicción de precios en la "app"

# Inflation_analysis

- Se utiliza "webscrapping" para obtener automáticamente la cifra del aumento por inflación respecto a 2017, fecha en que se compuso el "dataframe" original de precios (página: https://www.in2013dollars.com/Jewelry/price-inflation/2017-to-2023?amount=326)

- Por ahora, respecto a 2017, los precios han subido cerca de un 11%

- Se implementará este método en la "app"

- Se detecta también una alternativa para aproximar la inflación mediante la media de los últimos años, en caso de que la página dejara de funcionar puntualmente

# Images_data_processing

- En el "notebook" homónimo, se tratan varios "datasets" que representan un conjunto de características de imágenes de diamantes. 

- El proceso es el siguiente:

1) Se unen los "datasets" en un solo "dataframe"

2) Mediante diversas técnicas, se adapta el "dataframe" resultante para que contenga los mismos datos que requiere el otro modelo para hacer su predicción de los precios. No se puede disponer de la variable "table", sin embargo, así que el modelo que se use para predecir los precios a partir de las características obtenidas de las imágenes deberá haber sido entrenado sin esa variable

# Images_processing